In [17]:
import pandas as pd
from google.cloud import storage
from google.cloud import storage
import pandas as pd
from io import BytesIO
from google.cloud import bigquery
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.preprocessing import RobustScaler
%run functions.ipynb

In [18]:
from google.cloud import storage
import pandas as pd
from io import BytesIO

# Your GCS bucket name
bucket_name = 'ccai-storage'

# Paths to the Parquet files in your GCS bucket
articles_parquet_path = 'pipeline/final_df1.parquet'

# Initialize the GCS client
client = storage.Client()

# Load the Parquet files
final_df1 = load_parquet_from_gcs(bucket_name, articles_parquet_path)

In [19]:
print(final_df1['split'].value_counts())

split
TRAIN       2408
TEST         958
VALIDATE     956
Name: count, dtype: int64


In [21]:
final_df1

,uri,url,title,body,date,time,dateTime,dateTimePub,lang,isDuplicate,...,alexaGlobalRank,alexaCountryRank,date_column,year,month,year_month,topic,split,shares_scaled,body_pre
0,7582830489,https://www.ilfattoquotidiano.it/2023/06/07/pr...,"Premio Strega 2023, quattro scrittrici in fina...",Quattro scrittrici in cinquina alla finale del...,2023-06-07,19:32:00,2023-06-07T19:32:00Z,2023-06-07T19:29:00Z,ita,False,...,3336,47,2023-06-07,2023,6,2023-6,Arts and Entertainment,TRAIN,0.36,Quattro scrittrici cinquina finale Premio Stre...
1,7358061043,https://www.corriere.it/spettacoli/23_gennaio_...,La rivista Usa The Atlantic stronca i Maneskin...,Il giornalista Spencer Kornhaber fa a pezzi Da...,2023-01-19,18:21:00,2023-01-19T18:21:00Z,2023-01-19T18:10:00Z,ita,False,...,1633,30,2023-01-19,2023,1,2023-1,Arts and Entertainment,TRAIN,6.44,giornalista Spencer Kornhaber fa pezzi Damiano...
2,7376407793,https://www.tgcom24.mediaset.it/people/damiano...,"Damiano David, shopping in gioielleria con Gio...",Per Giorgia Soleri (e Damiano) la gravidanza p...,2023-01-31,09:44:00,2023-01-31T09:44:00Z,2023-01-31T09:43:00Z,ita,False,...,1969,28,2023-01-31,2023,1,2023-1,Arts and Entertainment,TRAIN,0.84,Giorgia Soleri ( Damiano ) gravidanza può aspe...
3,7807122008,https://www.ilfattoquotidiano.it/2023/10/29/x-...,"X Factor, Francesca Michielin e Ambra Angiolin...","Ieri sera, sabato 28 ottobre, Francesca Michie...",2023-10-29,11:03:39,2023-10-29T11:03:39Z,2023-10-29T11:00:59Z,ita,False,...,3336,47,2023-10-29,2023,10,2023-10,Arts and Entertainment,TRAIN,0.36,"Ieri serum , sabato 28 ottobre , Francesca Mic..."
4,7539580387,https://www.corriere.it/spettacoli/23_maggio_1...,Alessandro Cecchi Paone e Simone Antolini hann...,Ancora sconosciute le motivazioni della scelta...,2023-05-11,15:43:00,2023-05-11T15:43:00Z,2023-05-11T15:34:00Z,ita,False,...,1633,30,2023-05-11,2023,5,2023-5,Arts and Entertainment,TRAIN,0.92,Ancora sconosciute motivazioni scelta . celebr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4317,2023-12-196699085,https://www.repubblica.it/politica/2023/12/19/...,"Azzariti: ""Con il premierato vogliono ridurre ...","Il professore di Diritto costituzionale: ""Le p...",2023-12-19,01:28:03,2023-12-19T01:28:03Z,2023-12-19T00:05:00Z,ita,False,...,1376,8,2023-12-19,2023,12,2023-12,Politics,TEST,0.44,professore Diritto costituzionale : `` parole ...
4318,2023-12-196684238,https://www.repubblica.it/esteri/2023/12/18/ne...,"Gli Usa a Netanyahu: ""Proteggere i civili è un...",Il capo del Pentagono Austin in Israele. Galla...,2023-12-19,01:00:29,2023-12-19T01:00:29Z,2023-12-19T00:00:00Z,ita,False,...,1376,8,2023-12-19,2023,12,2023-12,Politics,TEST,-0.16,capo Pentagono Austin Israele . Gallant apre :...
4319,2023-12-196668947,https://www.repubblica.it/sport/calcio/serie-a...,"La Roma gela Mourinho: altro che rinnovo, prim...",L'apertura del tecnico a restare oltre il 2024...,2023-12-19,00:29:43,2023-12-19T00:29:43Z,2023-12-19T00:00:00Z,ita,False,...,1376,8,2023-12-19,2023,12,2023-12,Sports,TEST,0.28,L'apertura tecnico restare oltre 2024 sbatte F...
4320,2023-12-196655464,https://www.lastampa.it/sport/calcio/2023/12/1...,"Massimo Mauro: ""Un anno senza il mio amico Luc...","""No, vai"".\n\n""Va bene, passo il Natale a casa...",2023-12-19,00:03:20,2023-12-19T00:03:20Z,2023-12-19T00:00:00Z,ita,False,...,5616,91,2023-12-19,2023,12,2023-12,Sports,TEST,-0.16,"`` No , vai '' . `` Va bene , passo Natale cas..."


### ---------------------------------------------- Costi ---------------------------------------

In [12]:
##### Testiamo il costo della sentiment
character_count = final_df1['body_pre'].str.len()
total_character_count = character_count.sum()
total_character_count

10421551

In [11]:
total_character_mean = character_count.mean()
total_character_mean

2411.279731605738

In [14]:
total_character_mean*40000

96451189.26422952

#### Versione semplificata dataset

In [13]:
costo_sentiment_40karticles = tot_sample=(total_character_mean*40000/1000)*0.000250
costo_sentiment_40karticles

24.11279731605738

In [ ]:
#### --------------------------------

#### -------------------------------------------------------------------------
### Testiamo il sentiment

In [22]:
import pandas as pd
from google.cloud import language_v1
import json

def analyze_text_sentiment(row, text_column: str = 'preprocessed_text', num_documents: int = 10) -> dict:
    client = language_v1.LanguageServiceClient()
    text = row[text_column]
    document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)
    response = client.analyze_sentiment(request={'document': document})

    # Restituisce un dizionario con i risultati dell'analisi del sentiment
    return {
        'score': response.document_sentiment.score,
        'magnitude': response.document_sentiment.magnitude,
        'num_documents': num_documents
    }

num_documents=10
# Applica la funzione analyze_text_sentiment a ciascuna riga del DataFrame
sentiment_results = final_df1.head(num_documents).apply(lambda row: analyze_text_sentiment(row, text_column='body_pre', num_documents=num_documents), axis=1)

# Crea un DataFrame dai risultati dell'analisi del sentiment
df_sentiment = pd.DataFrame(sentiment_results.tolist())

# Unisce il DataFrame originale con i risultati dell'analisi del sentiment
df_sentiment = pd.concat([final_df1.head(num_documents), df_sentiment], axis=1)

### Salvo il df come parquet to the bucket
bucket_name="ccai-storage"

folder = "pipeline"
# Define your Google Cloud Storage bucket name and Parquet file name
parquet_file_name = 'final_df2.parquet'

# Save DataFrames to GCS as Parquet files in the 'fabio' folder
save_df_to_gcs_parquet(df_sentiment, bucket_name, folder, parquet_file_name)

#df_no_dups_test
# Remove the local Parquet file
# os.remove(parquet_file_name)



In [23]:
df_sentiment

,uri,url,title,body,date,time,dateTime,dateTimePub,lang,isDuplicate,...,year,month,year_month,topic,split,shares_scaled,body_pre,score,magnitude,num_documents
0,7582830489,https://www.ilfattoquotidiano.it/2023/06/07/pr...,"Premio Strega 2023, quattro scrittrici in fina...",Quattro scrittrici in cinquina alla finale del...,2023-06-07,19:32:00,2023-06-07T19:32:00Z,2023-06-07T19:29:00Z,ita,False,...,2023,6,2023-6,Arts and Entertainment,TRAIN,0.36,Quattro scrittrici cinquina finale Premio Stre...,0.1,7.5,10
1,7358061043,https://www.corriere.it/spettacoli/23_gennaio_...,La rivista Usa The Atlantic stronca i Maneskin...,Il giornalista Spencer Kornhaber fa a pezzi Da...,2023-01-19,18:21:00,2023-01-19T18:21:00Z,2023-01-19T18:10:00Z,ita,False,...,2023,1,2023-1,Arts and Entertainment,TRAIN,6.44,giornalista Spencer Kornhaber fa pezzi Damiano...,0.0,0.7,10
2,7376407793,https://www.tgcom24.mediaset.it/people/damiano...,"Damiano David, shopping in gioielleria con Gio...",Per Giorgia Soleri (e Damiano) la gravidanza p...,2023-01-31,09:44:00,2023-01-31T09:44:00Z,2023-01-31T09:43:00Z,ita,False,...,2023,1,2023-1,Arts and Entertainment,TRAIN,0.84,Giorgia Soleri ( Damiano ) gravidanza può aspe...,0.3,2.9,10
3,7807122008,https://www.ilfattoquotidiano.it/2023/10/29/x-...,"X Factor, Francesca Michielin e Ambra Angiolin...","Ieri sera, sabato 28 ottobre, Francesca Michie...",2023-10-29,11:03:39,2023-10-29T11:03:39Z,2023-10-29T11:00:59Z,ita,False,...,2023,10,2023-10,Arts and Entertainment,TRAIN,0.36,"Ieri serum , sabato 28 ottobre , Francesca Mic...",0.0,11.4,10
4,7539580387,https://www.corriere.it/spettacoli/23_maggio_1...,Alessandro Cecchi Paone e Simone Antolini hann...,Ancora sconosciute le motivazioni della scelta...,2023-05-11,15:43:00,2023-05-11T15:43:00Z,2023-05-11T15:34:00Z,ita,False,...,2023,5,2023-5,Arts and Entertainment,TRAIN,0.92,Ancora sconosciute motivazioni scelta . celebr...,0.1,1.2,10
5,7623352415,https://www.ilfattoquotidiano.it/2023/07/04/de...,"Dentro la grotta del ""Bueno Fonteno"": così i s...",Sono in corso le operazioni di recupero della ...,2023-07-04,08:31:00,2023-07-04T08:31:00Z,2023-07-04T08:30:00Z,ita,False,...,2023,7,2023-7,Arts and Entertainment,TRAIN,-0.24,corso operazioni recupero speleologa bloccata ...,-0.1,1.1,10
6,7820242200,https://www.corriere.it/alessandro-d-avenia-ul...,L&#8217;atomica che abbiamo nel cuore &#124; A...,Il male assedia le nostre vite. Una guerra in ...,2023-11-06,06:02:47,2023-11-06T06:02:47Z,2023-11-06T06:01:01Z,ita,False,...,2023,11,2023-11,Arts and Entertainment,TRAIN,7.08,male assedia vite . guerra seno all'Europa app...,0.0,4.4,10
7,7580215994,https://www.ilfattoquotidiano.it/2023/06/06/el...,"Elisa canta ""Se piovesse il tuo nome"" sotto il...",Elisa continua il suo live incurante della pio...,2023-06-06,09:56:00,2023-06-06T09:56:00Z,2023-06-06T09:32:00Z,ita,False,...,2023,6,2023-6,Arts and Entertainment,TRAIN,1.68,Elisa continuum live incurante pioggia : fan l...,0.3,5.0,10
8,7555212979,https://www.corriere.it/sport/calcio/serie-a/2...,Spalletti e De Laurentiis&#44; i motivi della ...,Non è il contratto lungo o corto ad aver inter...,2023-05-21,15:11:00,2023-05-21T15:11:00Z,2023-05-21T15:09:00Z,ita,False,...,2023,5,2023-5,Arts and Entertainment,TRAIN,0.04,contratto lungo corto aver interrotto rapporti...,0.0,4.3,10
9,7350803258,https://www.ilfattoquotidiano.it/2023/01/15/ro...,"Roma, Martina Scialdone uccisa a 35 anni dall'...","La vittima era un'avvocatessa di 35 anni, lui ...",2023-01-15,11:12:00,2023-01-15T11:12:00Z,2023-01-15T11:09:00Z,ita,False,...,2023,1,2023-1,Arts and Entertainment,TRAIN,1.28,"vittima un'avvocatessa 35 anni , ingegnere , s...",-0.2,9.7,10


In [ ]:
### Save df to parquet to the bucket

In [16]:
### Salvo il df come parquet to the bucket
bucket_name="ccai-storage"

folder = "pipeline"
# Define your Google Cloud Storage bucket name and Parquet file name
parquet_file_name = 'final_df2.parquet'

# Save DataFrames to GCS as Parquet files in the 'fabio' folder
save_df_to_gcs_parquet(df_sentiment, bucket_name, folder, parquet_file_name)

#df_no_dups_test
# Remove the local Parquet file
# os.remove(parquet_file_name)

In [ ]:
### 

### Fine seconda parte